<a href="https://colab.research.google.com/github/psriraj17/sentiment_analysis/blob/main/sentiment_analysis_NB_nltk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import io
import numpy as np
import re
import pickle
import nltk
import string
from nltk.tokenize import RegexpTokenizer
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn
from sklearn import naive_bayes

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving train-v2.tsv to train-v2 (1).tsv


In [ ]:
train_df= pd.read_csv(io.BytesIO(uploaded['train-v2.tsv']), sep='\t',header=None)
train_df

,0,1
0,0,@USER @USER a sicrhau bod mwy o arian poced 'd...
1,1,Parti Dolig da gyda tim swyddfa canolog @USER ...
2,0,@USER yeaah ma fe yn wir. ( oh well.
3,1,@USER hahaha idk. 3am oedd y bws ti?
4,0,@USER dwim yn gal llun ohoni?
...,...,...
78604,1,"@USER @USER diolch yn fawr am ein diweddaru, r..."
78605,1,@USER @USER @USER diolch! Digon o opsiynne felly!
78606,0,mor hungover heddiw 🔫
78607,1,"@USER eidiaaaal, awni dydd llun nesa ia? Y air..."


In [ ]:
train_df.rename(columns={0: 'Target', 1: 'Text'}, inplace=True)
train_df

,Target,Text
0,0,@USER @USER a sicrhau bod mwy o arian poced 'd...
1,1,Parti Dolig da gyda tim swyddfa canolog @USER ...
2,0,@USER yeaah ma fe yn wir. ( oh well.
3,1,@USER hahaha idk. 3am oedd y bws ti?
4,0,@USER dwim yn gal llun ohoni?
...,...,...
78604,1,"@USER @USER diolch yn fawr am ein diweddaru, r..."
78605,1,@USER @USER @USER diolch! Digon o opsiynne felly!
78606,0,mor hungover heddiw 🔫
78607,1,"@USER eidiaaaal, awni dydd llun nesa ia? Y air..."


In [ ]:
# Storing data in lists.
target, text = list(train_df['Target']), list(train_df['Text'])

In [ ]:
df_positive = train_df[train_df['Target'] == 1]
df_negative = train_df[train_df['Target'] == 0]

In [ ]:
pos_list=list(df_positive['Text'])
neg_list=list(df_negative['Text'])

In [ ]:
username=['@USER']

In [ ]:
#This piece of code was taken from https://www.analyticsvidhya.com/blog/2021/06/twitter-sentiment-analysis-a-nlp-use-case-for-beginners/
def cleaning_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in username])
df_positive['Text'] = df_positive['Text'].apply(lambda text: cleaning_stopwords(text))
df_positive['Text'].head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


1     Parti Dolig da gyda tim swyddfa canolog Ty Gwy...
3                        hahaha idk. 3am oedd y bws ti?
6          Diolch! Dweud wrtho Manon fi wedi txto hi! x
8     mor browd, fe fyddai yn dangos ti faint mor aw...
10    ond paid a meiddio darllen y tweet yma allan a...
Name: Text, dtype: object

In [ ]:
def cleaning_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in username])
df_negative['Text'] = df_negative['Text'].apply(lambda text: cleaning_stopwords(text))
df_negative['Text'].head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


0    a sicrhau bod mwy o arian poced 'da ti, nes i ...
2                       yeaah ma fe yn wir. ( oh well.
4                              dwim yn gal llun ohoni?
5    desparate housewives! Dadla be di gora heno, g...
7    mar boy yn hymian o nappies. Dwim isho siarad ...
Name: Text, dtype: object

In [ ]:
punct_list= string.punctuation
def cleaning_punctuations(text):
    translator = str.maketrans('', '', punct_list)
    return text.translate(translator)
df_positive['Text']= df_positive['Text'].apply(lambda x: cleaning_punctuations(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
punct_list=string.punctuation
def cleaning_punctuations(text):
    translator = str.maketrans('', '', punct_list)
    return text.translate(translator)
df_negative['Text']= df_negative['Text'].apply(lambda x: cleaning_punctuations(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
df_positive = df_positive.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))
df_negative = df_negative.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))

In [ ]:
pos_list=list(df_positive['Text'])
neg_list=list(df_positive['Text'])
df_positive['Text'] = df_positive['Text'].str.replace('\n', '')
df_positive['Text'] = df_positive['Text'].str.replace('\t', ' ')
df_positive['Text'] = df_positive['Text'].str.replace(' {2,}', ' ', regex=True)
df_positive['Text'] = df_positive['Text'].str.strip()
df_negative['Text'] = df_negative['Text'].str.replace('\n', ' ')
df_negative['Text'] = df_negative['Text'].str.replace('\t', ' ')
df_negative['Text'] = df_negative['Text'].str.replace(' {2,}', ' ', regex=True)
df_negative['Text'] = df_negative['Text'].str.strip()
df_positive['Text'] = df_positive['Text'].drop_duplicates()
df_negative['Text'] = df_negative['Text'].drop_duplicates()
df_positive['Text'] = df_positive['Text'].str.lower()
df_negative['Text'] = df_negative['Text'].str.lower()
combine_df=pd.concat([df_positive, df_negative])
combine_df=combine_df.fillna('')
combine_df

,Target,Text
1,1,parti dolig da gyda tim swyddfa canolog ty gwy...
3,1,hahaha idk 3am oedd y bws ti
6,1,diolch dweud wrtho manon fi wedi txto hi x
8,1,mor browd fe fyddai yn dangos ti faint mor awe...
10,1,ond paid a meiddio darllen y tweet yma allan a...
...,...,...
78596,0,fina fyd tin sgol fory sgenaim mynadd o gwbwl de
78597,0,all white wash a playsuit gwyn fo tiny bit o p...
78601,0,cymryd un fi lawr fory
78606,0,mor hungover heddiw


In [ ]:
vectorizer = TfidfVectorizer(lowercase= True, strip_accents= 'ascii')

In [ ]:
y = combine_df.Target
x= vectorizer.fit_transform(combine_df.Text)

,Target,Text
1,1,parti dolig da gyda tim swyddfa canolog ty gwy...
3,1,hahaha idk 3am oedd y bws ti
6,1,diolch dweud wrtho manon fi wedi txto hi x
8,1,mor browd fe fyddai yn dangos ti faint mor awe...
10,1,ond paid a meiddio darllen y tweet yma allan a...
...,...,...
78596,0,fina fyd tin sgol fory sgenaim mynadd o gwbwl de
78597,0,all white wash a playsuit gwyn fo tiny bit o p...
78601,0,cymryd un fi lawr fory
78606,0,mor hungover heddiw


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y, random_state= 42)


In [ ]:
clf = naive_bayes.MultinomialNB()
clf.fit(x_train, y_train)

MultinomialNB()

In [ ]:

sklearn.metrics.roc_auc_score(y_test, clf.predict_proba(x_test)[:,1])

0.8075054046958587